### SNU 2021-1 Introduction to Artificial Intelligence 실습

In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import datasets
from torchvision import transforms as T
import matplotlib.pyplot as plt

In [4]:
train_dset = datasets.MNIST(root='data', train=True, transform=T.ToTensor(), download=True)
test_dset = datasets.MNIST(root='data', train=False, transform=T.ToTensor(), download=True)

100.1%Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw
113.5%Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw
100.4%Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw
180.4%Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


In [22]:
train_loader = torch.utils.data.DataLoader(train_dset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dset, batch_size=16, shuffle=False)

In [38]:
def train(model, train_loader, optimizer, device):
    for i, (image, target) in enumerate(train_loader):
        image = image.to(device)
        output = model(image)
        loss = F.cross_entropy(output, target)

        model.zero_grad()
        loss.backward()
        optimizer.step()

        loss_values.append(loss.detach().numpy()) # np.ndarray

In [39]:
def test(model, test_loader, device):
    accurate = 0
    with torch.no_grad():
        for i, (image, target) in enumerate(test_loader):
            image = image.to(device)
            output = model(image)

            prediction = torch.argmax(output, dim=1)

            for p,t in zip(prediction, target):
                if p == t:
                    accurate += 1
    accuracy = accurate / len(test_loader.dataset)
    return accuracy

In [40]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()

        self.fc = nn.Sequential(
            nn.Linear(28*28, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        x = x.view(-1, 28*28) # batch_size, neurons of input layer
        return self.fc(x)

In [41]:
loss_values = []
accuracy_values = []

model = net()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for i_epoch in range(10):
    train(model, train_loader, optimizer, i_epoch, device)

test(model, test_loader, i_epoch, device)